In [1]:
import os
import gc
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor # MODIFIED: Import RandomForest
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
import joblib
import os
import warnings

In [2]:

import os
import gc
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
import joblib

# --- 1. Configuración General ---
PARQUET_FILE = '../data/gold_ventas_semanales_clustered_lgbm.parquet' # Ensure this path is correct
OUTPUT_DIR = '../models/random_forest'
METRICS_FILE = os.path.join(OUTPUT_DIR, 'all_cluster_metrics_rf.csv')
PREDICTIONS_FILE = os.path.join(OUTPUT_DIR, 'all_series_predictions_rf.csv')
PLOTS_DIR = os.path.join(OUTPUT_DIR, 'prediction_plots_by_series_rf')
MODELS_DIR = os.path.join(OUTPUT_DIR, 'trained_cluster_models_rf')

os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(PLOTS_DIR, exist_ok=True)
os.makedirs(MODELS_DIR, exist_ok=True)

TARGET_VARIABLE = 'weekly_volume'
SERIES_ID_COLS = ['establecimiento', 'material']
CLUSTER_COL = 'cluster_label'
DATE_COL = 'week'

# Leer metadata para obtener todas las columnas
try:
    parquet_meta = pq.read_metadata(PARQUET_FILE)
    ALL_COLS = [col.name for col in parquet_meta.schema]
except Exception as e:
    print(f"Error reading Parquet metadata from {PARQUET_FILE}: {e}. Please check the file path and integrity. Exiting.")
    exit()


FEATURES_COLS = [
    col for col in ALL_COLS
    if col not in [TARGET_VARIABLE, DATE_COL, CLUSTER_COL, 'last_sale_week']
]

CATEGORICAL_FEATURES_FOR_MODEL = [f for f in ['establecimiento', 'material'] if f in FEATURES_COLS]
NUMERICAL_FEATURES = [col for col in FEATURES_COLS if col not in CATEGORICAL_FEATURES_FOR_MODEL]

if not NUMERICAL_FEATURES and not CATEGORICAL_FEATURES_FOR_MODEL:
    print(f"Warning: No numerical or categorical features selected for the model based on FEATURES_COLS. FEATURES_COLS: {FEATURES_COLS}")
    print(f"NUMERICAL_FEATURES: {NUMERICAL_FEATURES}, CATEGORICAL_FEATURES_FOR_MODEL: {CATEGORICAL_FEATURES_FOR_MODEL}")
    # Potentially exit or handle this scenario if it's unexpected
    # exit()
elif not FEATURES_COLS:
    print(f"Warning: FEATURES_COLS is empty. No features to train on. Exiting.")
    # exit()


# --- 2. Funciones de Utilidad (Métricas y Gráficos) ---
def calculate_mase(y_true_train, y_true_test, y_pred_test):
    y_true_train = np.array(y_true_train).flatten()
    y_true_test = np.array(y_true_test).flatten()
    y_pred_test = np.array(y_pred_test).flatten()

    if len(y_true_train) < 2:
        return np.nan

    naive_forecast_error_train = np.mean(np.abs(np.diff(y_true_train)))

    if naive_forecast_error_train < 1e-9:
         model_mae_test = mean_absolute_error(y_true_test, y_pred_test)
         return np.inf if model_mae_test > 1e-9 else 0.0

    model_mae_test = mean_absolute_error(y_true_test, y_pred_test)
    return model_mae_test / naive_forecast_error_train


def evaluate_model(y_true_train, y_true_test, y_pred_test, label="Cluster"):
    metrics = {
        f'{label}_mae': mean_absolute_error(y_true_test, y_pred_test),
        f'{label}_rmse': np.sqrt(mean_squared_error(y_true_test, y_pred_test)),
        f'{label}_mape': mean_absolute_percentage_error(y_true_test, y_pred_test) if np.all(np.abs(y_true_test) > 1e-9) else np.nan,
        f'{label}_r2': r2_score(y_true_test, y_pred_test),
        f'{label}_mase': calculate_mase(y_true_train, y_true_test, y_pred_test)
    }
    mape_key = f'{label}_mape'
    if mape_key in metrics and np.isinf(metrics[mape_key]):
         metrics[mape_key] = np.nan
    return metrics


def plot_predictions(dates_test, y_true_test, y_pred_test, estab, material, cluster_id, filepath):
    plt.figure(figsize=(15, 6))
    plt.plot(dates_test, y_true_test, label='Real', marker='.', linestyle='-')
    plt.plot(dates_test, y_pred_test, label=f'Predicción RF (Cluster {cluster_id})', marker='x', linestyle='--')
    plt.title(f'Predicción RF vs Real - {estab} / {material} (Cluster {cluster_id})')
    plt.xlabel('Semana')
    plt.ylabel('Volumen Semanal')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(filepath)
    plt.close()

# --- 3. Carga Inicial de Clusters ---
unique_clusters = [] # Initialize
try:
    print(f"Leyendo columna de clusters '{CLUSTER_COL}' del archivo: {PARQUET_FILE}")
    pf = pq.ParquetFile(PARQUET_FILE)
    cluster_labels_df = pf.read(columns=[CLUSTER_COL]).to_pandas()
    unique_clusters = cluster_labels_df[CLUSTER_COL].unique()
    unique_clusters = [c for c in unique_clusters if pd.notna(c) and c != ''] # Handle empty strings as well if they act like NaN
    print(f"Encontrados {len(unique_clusters)} clusters únicos: {unique_clusters[:10]}...") # Print a sample
    del cluster_labels_df
    gc.collect()
except Exception as e:
    print(f"Error fatal al leer la columna de clusters del Parquet: {e}. Verifique el archivo y la columna '{CLUSTER_COL}'. Exiting.")
    exit()

if not unique_clusters:
    print(f"No se encontraron clusters válidos en la columna '{CLUSTER_COL}' del archivo '{PARQUET_FILE}'. Verifique los datos. Exiting.")
    exit()

# --- 4. Procesamiento por Cluster (Entrenamiento, CV, HPT, Evaluación) ---
all_cluster_metrics = []
header_preds = SERIES_ID_COLS + [DATE_COL, 'actual_volume', 'predicted_volume', CLUSTER_COL]
pd.DataFrame(columns=header_preds).to_csv(PREDICTIONS_FILE, index=False)

N_SPLITS_CV = 5
N_ITER_HPT = 5 # Adjust for speed vs. thoroughness
SCORING_METRIC_HPT = 'neg_mean_absolute_error'

param_dist_rf = {
    'regressor__n_estimators': [50, 100, 150], # Reduced for quicker example runs
    'regressor__max_depth': [None, 10, 20],
    'regressor__min_samples_split': [10, 20, 100],
    'regressor__min_samples_leaf': [5, 10, 50],
    'regressor__max_features': ['sqrt', 'log2', 0.7]
}

header_metrics = [CLUSTER_COL] + [f'Cluster_{m}' for m in ['mae', 'rmse', 'mape', 'r2', 'mase']] + ['best_params']
pd.DataFrame(columns=header_metrics).to_csv(METRICS_FILE, index=False)


print(f"\nIniciando procesamiento para {len(unique_clusters)} clusters...")
for cluster_count, cluster_id in enumerate(unique_clusters):
    print(f"\n--- Procesando Cluster {cluster_count+1}/{len(unique_clusters)}: ID = {cluster_id} ---")

    cluster_df = None
    X_cluster, y_cluster, ids_cluster, dates_cluster = None, None, None, None
    X_train_val, X_test, y_train_val, y_test = None, None, None, None
    ids_test, dates_test = None, None
    pipeline, search, best_model_cluster = None, None, None
    y_pred_test_cluster = None
    predictions_df = None

    try:
        print(f"Cargando datos para el cluster {cluster_id}...")
        filters = [(CLUSTER_COL, '=', cluster_id)]
        cluster_df = pd.read_parquet(PARQUET_FILE, filters=filters)
        cluster_df = cluster_df.sort_values(by=SERIES_ID_COLS + [DATE_COL]).reset_index(drop=True)
        print(f"Cluster {cluster_id}: {len(cluster_df)} filas cargadas.")

        if len(cluster_df) < (N_SPLITS_CV + 2) * 2 :
            print(f"Datos insuficientes para CV en cluster {cluster_id} ({len(cluster_df)} filas). Saltando cluster.")
            continue
        # Define a threshold and a sampling size
        VERY_LARGE_CLUSTER_THRESHOLD = 2000000 # e.g., if more than 2M rows
        SAMPLE_SIZE_FOR_LARGE_CLUSTERS = 500000 # Target approximate sample size

        # --- MODIFIED SUBSAMPLING LOGIC ---
        # Apply this subsampling if the cluster is very large (e.g., cluster_id == 2 or general threshold)
        # if cluster_id == 2 and len(cluster_df) > VERY_LARGE_CLUSTER_THRESHOLD: # Example for a specific cluster
        if len(cluster_df) > VERY_LARGE_CLUSTER_THRESHOLD: # General threshold for any large cluster
            print(f"Cluster {cluster_id} is very large ({len(cluster_df)} rows). Subsampling complete series to be around {SAMPLE_SIZE_FOR_LARGE_CLUSTERS} rows.")

            unique_series_identifiers = cluster_df[SERIES_ID_COLS].drop_duplicates()
            n_unique_series_original = len(unique_series_identifiers)

            if n_unique_series_original == 0:
                print(f"Warning: No unique series found in cluster {cluster_id} for subsampling. Skipping subsampling logic.")
            elif n_unique_series_original == 1:
                print(f"Cluster {cluster_id} has only one series ({len(cluster_df)} rows). If it's too large, consider other strategies like truncating the series or feature engineering for scalability.")
                # If this single series is still too large, you might need to truncate it, e.g., take the last N points
                if len(cluster_df) > SAMPLE_SIZE_FOR_LARGE_CLUSTERS:
                    print(f"The single series in cluster {cluster_id} has {len(cluster_df)} rows, which is more than the target {SAMPLE_SIZE_FOR_LARGE_CLUSTERS}. Truncating to the most recent {SAMPLE_SIZE_FOR_LARGE_CLUSTERS} observations for this series.")
                    # Ensure it's sorted by date to get the most recent
                    cluster_df = cluster_df.sort_values(by=SERIES_ID_COLS + [DATE_COL])
                    cluster_df = cluster_df.tail(SAMPLE_SIZE_FOR_LARGE_CLUSTERS).reset_index(drop=True)

            else:
                # Estimate average rows per series to guide how many series to sample
                avg_rows_per_series = len(cluster_df) / n_unique_series_original
                
                # Calculate how many series to sample to get *approximately* SAMPLE_SIZE_FOR_LARGE_CLUSTERS rows
                # This is an estimate; the actual number of rows will vary.
                num_series_to_sample = int(SAMPLE_SIZE_FOR_LARGE_CLUSTERS / avg_rows_per_series)
                num_series_to_sample = max(1, min(num_series_to_sample, n_unique_series_original)) # Ensure it's within bounds

                print(f"Original series count: {n_unique_series_original}. Estimated series to sample: {num_series_to_sample}.")
                
                # Randomly select series identifiers
                sampled_series_ids = unique_series_identifiers.sample(n=num_series_to_sample, random_state=42)
                
                # Filter the original DataFrame to include only the rows from the sampled series
                # Using merge is a robust way to do this
                cluster_df = pd.merge(cluster_df, sampled_series_ids, on=SERIES_ID_COLS, how='inner')
                
                # After merging, it's good practice to re-sort if the order might have changed,
                # though 'inner' merge should preserve order within groups from the left DataFrame.
                # Explicitly re-sorting ensures data is correctly ordered for time series operations.
                cluster_df = cluster_df.sort_values(by=SERIES_ID_COLS + [DATE_COL]).reset_index(drop=True)
                
                print(f"Cluster {cluster_id} now has {len(cluster_df)} rows from {len(sampled_series_ids)} unique series after subsampling.")

                # Optional: If still too large (because some sampled series were huge), you could add a second pass
                # to trim the largest series or further remove some of the sampled series.
                # For now, this provides a good balance.
        # --- END OF MODIFIED SUBSAMPLING LOGIC ---


        X_cluster = cluster_df[FEATURES_COLS]
        y_cluster = cluster_df[TARGET_VARIABLE]
        ids_cluster = cluster_df[SERIES_ID_COLS + [CLUSTER_COL]]
        dates_cluster = cluster_df[DATE_COL]

        numeric_transformer = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ])
        categorical_transformer = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
            ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=True)) # MODIFIED
        ])
        preprocessor = ColumnTransformer(
            transformers=[
                ('num', numeric_transformer, NUMERICAL_FEATURES),
                ('cat', categorical_transformer, CATEGORICAL_FEATURES_FOR_MODEL)
            ],
            remainder='passthrough'
        )

        print(f"Realizando split Train/Test para el cluster {cluster_id}...")
        test_size_ratio = 0.2
        n_rows = len(cluster_df)
        split_point = int(n_rows * (1 - test_size_ratio))

        if split_point < N_SPLITS_CV + 1 or n_rows - split_point < 1:
             print(f"Datos insuficientes para split Train/Test adecuado en cluster {cluster_id}. Saltando.")
             continue

        X_train_val, X_test = X_cluster.iloc[:split_point], X_cluster.iloc[split_point:]
        y_train_val, y_test = y_cluster.iloc[:split_point], y_cluster.iloc[split_point:]
        ids_test = ids_cluster.iloc[split_point:]
        dates_test = dates_cluster.iloc[split_point:]
        y_true_train_for_mase = y_train_val.copy().values
        print(f"Cluster {cluster_id}: Train/Val={len(X_train_val)}, Test={len(X_test)}")

        cv_test_size = max(1, len(X_train_val) // (N_SPLITS_CV + 1))
        tscv = TimeSeriesSplit(n_splits=N_SPLITS_CV, gap=0, test_size=cv_test_size)

        rf_model = RandomForestRegressor(random_state=42, n_jobs=1
                                         )
        pipeline = Pipeline([
            ('preprocess', preprocessor),
            ('regressor', rf_model)
        ])

        search = RandomizedSearchCV(
            estimator=pipeline,
            param_distributions=param_dist_rf,
            n_iter=N_ITER_HPT,
            cv=tscv,
            scoring=SCORING_METRIC_HPT,
            n_jobs=1, # Use 1 for RandomizedSearchCV to avoid issues with nested parallelism if RF also uses n_jobs
            refit=True,
            random_state=42,
            verbose=1
        )
        print("!!! DIAGNOSTIC: Starting manual pipeline fit test on a TINY sample...")
        if 'X_train_val' in locals() and 'y_train_val' in locals() and not X_train_val.empty:
            # Ensure X_train_val and y_train_val are Pandas DataFrames/Series for .head()
            if not isinstance(X_train_val, pd.DataFrame):
                print("Warning: X_train_val is not a Pandas DataFrame, .head() might fail or behave unexpectedly.")
            if not isinstance(y_train_val, pd.Series):
                print("Warning: y_train_val is not a Pandas Series, .head() might fail or behave unexpectedly.")

            X_sample_diag = X_train_val.head(1000) # Use a very small sample, e.g., 1000 rows
            y_sample_diag = y_train_val.head(1000)
            print(f"Sampled {len(X_sample_diag)} rows for diagnostic fit.")

            # 'pipeline' should be the Pipeline object you're passing to RandomizedSearchCV
            # It contains your preprocessor and the RandomForestRegressor
            # Ensure 'pipeline' is defined correctly before this point.
            # Example if you need to redefine for clarity (but use your actual pipeline):
            # from sklearn.ensemble import RandomForestRegressor
            # rf_model_diag = RandomForestRegressor(random_state=42, n_jobs=2) # Or n_jobs=1
            # pipeline_diag_instance = Pipeline([
            #     ('preprocess', preprocessor), # 'preprocessor' is your ColumnTransformer
            #     ('regressor', rf_model_diag)
            # ])

            try:
                import time
                start_time_diag = time.time()
                print("!!! DIAGNOSTIC: Calling pipeline.fit() on sample...")
                pipeline.fit(X_sample_diag, y_sample_diag) # Use your actual 'pipeline' variable
                end_time_diag = time.time()
                print(f"!!! DIAGNOSTIC: Manual pipeline fit on sample completed in {end_time_diag - start_time_diag:.2f} seconds.")
            except Exception as e_diag:
                print(f"!!! DIAGNOSTIC ERROR during manual pipeline fit on sample: {e_diag}")
                import traceback
                traceback.print_exc()
        else:
            print("!!! DIAGNOSTIC: X_train_val or y_train_val not available for sample fitting test.")
        print("!!! DIAGNOSTIC: End of manual test. Continuing to RandomizedSearchCV (or not, if you exit)...")
        # You might want to add exit() here during testing to not proceed to the full RandomizedSearchCV
        # import sys
        # sys.exit("Exiting after diagnostic test.")
        # --- End of Temporary Diagnostic Code ---


        print(f"Iniciando HPT con RandomForest para cluster {cluster_id}...")
        search.fit(X_train_val, y_train_val)
        best_model_cluster = search.best_estimator_
        best_params_cluster = search.best_params_
        print(f"Mejores parámetros encontrados para cluster {cluster_id} (RandomForest): {best_params_cluster}")

        print(f"Realizando predicciones en el conjunto de test del cluster {cluster_id}...")
        y_pred_test_cluster = best_model_cluster.predict(X_test)

        print(f"Calculando métricas agregadas para el cluster {cluster_id}...")
        cluster_metrics = evaluate_model(y_true_train_for_mase, y_test.values, y_pred_test_cluster, label="Cluster")
        metrics_row = {CLUSTER_COL: cluster_id}
        metrics_row.update(cluster_metrics)
        metrics_row['best_params'] = str(best_params_cluster)
        all_cluster_metrics.append(metrics_row)
        pd.DataFrame([metrics_row]).to_csv(METRICS_FILE, mode='a', header=False, index=False)

        predictions_df = pd.DataFrame({
            'establecimiento': ids_test['establecimiento'],
            'material': ids_test['material'],
            DATE_COL: dates_test,
            'actual_volume': y_test.values,
            'predicted_volume': y_pred_test_cluster,
            CLUSTER_COL: ids_test[CLUSTER_COL]
        })
        predictions_df.to_csv(PREDICTIONS_FILE, mode='a', header=False, index=False)

        print(f"Generando gráficos para las series del cluster {cluster_id}...")
        unique_series_in_test_df = ids_test[SERIES_ID_COLS].drop_duplicates()
        max_plots_per_cluster = 100
        plot_count = 0
        for _, series_row in unique_series_in_test_df.iterrows():
            if plot_count >= max_plots_per_cluster:
                print(f"Límite de {max_plots_per_cluster} gráficos alcanzado para el cluster {cluster_id}.")
                break
            estab = series_row['establecimiento']
            material = series_row['material']
            mask = (predictions_df['establecimiento'] == estab) & \
                   (predictions_df['material'] == material)
            series_pred_df = predictions_df[mask]
            if not series_pred_df.empty:
                plot_filename = os.path.join(PLOTS_DIR, f'pred_vs_actual_rf_{estab}_{material}_cluster{cluster_id}.png')
                plot_predictions(
                    series_pred_df[DATE_COL],
                    series_pred_df['actual_volume'],
                    series_pred_df['predicted_volume'],
                    estab, material, cluster_id, plot_filename
                )
                plot_count +=1

        model_filename = os.path.join(MODELS_DIR, f'rf_model_cluster_{cluster_id}.joblib')
        joblib.dump(best_model_cluster, model_filename)
        print(f"Modelo RandomForest del cluster {cluster_id} guardado en: {model_filename}")

    except Exception as e:
        print(f"ERROR procesando el cluster {cluster_id}: {e}")
        import traceback
        traceback.print_exc() # Print full traceback for detailed error diagnosis
        error_row = {CLUSTER_COL: cluster_id}
        for m_key in header_metrics:
            if m_key not in [CLUSTER_COL, 'best_params']:
                error_row[m_key] = 'ERROR'
        error_row['best_params'] = str(e)
        pd.DataFrame([error_row]).to_csv(METRICS_FILE, mode='a', header=False, index=False)

    finally:
        del cluster_df, X_cluster, y_cluster, ids_cluster, dates_cluster
        del X_train_val, X_test, y_train_val, y_test, ids_test, dates_test
        del pipeline, search, best_model_cluster, y_pred_test_cluster, predictions_df
        gc.collect()

print("\n--- Proceso Completado ---")
# Final summary metrics can be loaded and printed here if desired
# df_metrics = pd.read_csv(METRICS_FILE)
# print(df_metrics.describe())
print(f"Resultados de RandomForest guardados en el directorio: {OUTPUT_DIR}")

Leyendo columna de clusters 'cluster_label' del archivo: ../data/gold_ventas_semanales_clustered_lgbm.parquet
Encontrados 7 clusters únicos: [np.int32(2), np.int32(0), np.int32(5), np.int32(6), np.int32(4), np.int32(3), np.int32(1)]...

Iniciando procesamiento para 7 clusters...

--- Procesando Cluster 1/7: ID = 2 ---
Cargando datos para el cluster 2...
Cluster 2: 16151645 filas cargadas.
Cluster 2 is very large (16151645 rows). Subsampling complete series to be around 500000 rows.
Original series count: 145068. Estimated series to sample: 4490.
Cluster 2 now has 500255 rows from 4490 unique series after subsampling.
Realizando split Train/Test para el cluster 2...
Cluster 2: Train/Val=400204, Test=100051
!!! DIAGNOSTIC: Starting manual pipeline fit test on a TINY sample...
Sampled 1000 rows for diagnostic fit.
!!! DIAGNOSTIC: Calling pipeline.fit() on sample...
!!! DIAGNOSTIC: Manual pipeline fit on sample completed in 0.73 seconds.
!!! DIAGNOSTIC: End of manual test. Continuing to Ra

KeyboardInterrupt: 

In [13]:
def run_random_forest_by_clusters(
    cluster_labels,
    max_series_per_cluster=500000,
    max_plots_per_cluster=100,
    parquet_file=PARQUET_FILE,
    output_dir=OUTPUT_DIR,
    metrics_file=METRICS_FILE,
    predictions_file=PREDICTIONS_FILE,
    plots_dir=PLOTS_DIR,
    models_dir=MODELS_DIR
):
    import gc
    import pandas as pd
    import numpy as np
    import pyarrow.parquet as pq
    from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
    from sklearn.pipeline import Pipeline
    from sklearn.preprocessing import StandardScaler, OneHotEncoder
    from sklearn.compose import ColumnTransformer
    from sklearn.impute import SimpleImputer
    import matplotlib.pyplot as plt
    import joblib
    import os

    # --- Configuración General ---
    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(plots_dir, exist_ok=True)
    os.makedirs(models_dir, exist_ok=True)

    TARGET_VARIABLE = 'weekly_volume'
    SERIES_ID_COLS = ['establecimiento', 'material']
    CLUSTER_COL = 'cluster_label'
    DATE_COL = 'week'

    # Leer metadata para obtener todas las columnas
    parquet_meta = pq.read_metadata(parquet_file)
    ALL_COLS = [col.name for col in parquet_meta.schema]

    FEATURES_COLS = [
        col for col in ALL_COLS
        if col not in [TARGET_VARIABLE, DATE_COL, CLUSTER_COL, 'last_sale_week']
    ]

    CATEGORICAL_FEATURES_FOR_MODEL = [f for f in ['establecimiento', 'material'] if f in FEATURES_COLS]
    NUMERICAL_FEATURES = [col for col in FEATURES_COLS if col not in CATEGORICAL_FEATURES_FOR_MODEL]

    def calculate_mase(y_true_train, y_true_test, y_pred_test):
        y_true_train = np.array(y_true_train).flatten()
        y_true_test = np.array(y_true_test).flatten()
        y_pred_test = np.array(y_pred_test).flatten()
        if len(y_true_train) < 2:
            return np.nan
        naive_forecast_error_train = np.mean(np.abs(np.diff(y_true_train)))
        if naive_forecast_error_train < 1e-9:
            model_mae_test = mean_absolute_error(y_true_test, y_pred_test)
            return np.inf if model_mae_test > 1e-9 else 0.0
        model_mae_test = mean_absolute_error(y_true_test, y_pred_test)
        return model_mae_test / naive_forecast_error_train

    def evaluate_model(y_true_train, y_true_test, y_pred_test, label="Cluster"):
        metrics = {
            f'{label}_mae': mean_absolute_error(y_true_test, y_pred_test),
            f'{label}_rmse': np.sqrt(mean_squared_error(y_true_test, y_pred_test)),
            f'{label}_mape': mean_absolute_percentage_error(y_true_test, y_pred_test) if np.all(np.abs(y_true_test) > 1e-9) else np.nan,
            f'{label}_r2': r2_score(y_true_test, y_pred_test),
            f'{label}_mase': calculate_mase(y_true_train, y_true_test, y_pred_test)
        }
        mape_key = f'{label}_mape'
        if mape_key in metrics and np.isinf(metrics[mape_key]):
            metrics[mape_key] = np.nan
        return metrics

    def plot_predictions(dates_test, y_true_test, y_pred_test, estab, material, cluster_id, filepath):
        plt.figure(figsize=(15, 6))
        plt.plot(dates_test, y_true_test, label='Real', marker='.', linestyle='-')
        plt.plot(dates_test, y_pred_test, label=f'Predicción RF (Cluster {cluster_id})', marker='x', linestyle='--')
        plt.title(f'Predicción RF vs Real - {estab} / {material} (Cluster {cluster_id})')
        plt.xlabel('Semana')
        plt.ylabel('Volumen Semanal')
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(filepath)
        plt.close()

    header_preds = SERIES_ID_COLS + [DATE_COL, 'actual_volume', 'predicted_volume', CLUSTER_COL]
    pd.DataFrame(columns=header_preds).to_csv(predictions_file, index=False)

    header_metrics = [CLUSTER_COL] + [f'Cluster_{m}' for m in ['mae', 'rmse', 'mape', 'r2', 'mase']] + ['best_params']
    pd.DataFrame(columns=header_metrics).to_csv(metrics_file, index=False)

    N_SPLITS_CV = 5
    N_ITER_HPT = 5
    SCORING_METRIC_HPT = 'neg_mean_absolute_error'
    param_dist_rf = {
        'regressor__n_estimators': [50, 100, 150],
        'regressor__max_depth': [None, 10, 20],
        'regressor__min_samples_split': [10, 20, 100],
        'regressor__min_samples_leaf': [5, 10, 50],
        'regressor__max_features': ['sqrt', 'log2', 0.7]
    }

    print(f"\nIniciando procesamiento para {len(cluster_labels)} clusters...")
    for cluster_count, cluster_id in enumerate(cluster_labels):
        print(f"\n--- Procesando Cluster {cluster_count+1}/{len(cluster_labels)}: ID = {cluster_id} ---")
        try:
            print(f"Cargando datos para el cluster {cluster_id}...")
            filters = [(CLUSTER_COL, '=', cluster_id)]
            cluster_df = pd.read_parquet(parquet_file, filters=filters)
            cluster_df = cluster_df.sort_values(by=SERIES_ID_COLS + [DATE_COL]).reset_index(drop=True)
            print(f"Cluster {cluster_id}: {len(cluster_df)} filas cargadas.")
            # Limitar a max_series_per_cluster series únicas
            unique_series = cluster_df[SERIES_ID_COLS].drop_duplicates()

            if len(cluster_df) > max_series_per_cluster:
                print(f"Cluster {cluster_id} is very large ({len(cluster_df)} rows). Subsampling complete series to be around {max_series_per_cluster} rows.")
                unique_series = cluster_df[SERIES_ID_COLS].drop_duplicates()
                n_unique_series_original = len(unique_series)
                if n_unique_series_original == 0:
                    print(f"Warning: No unique series found in cluster {cluster_id} for subsampling. Skipping subsampling logic.")
                elif n_unique_series_original == 1:
                    print(f"Cluster {cluster_id} has only one series ({len(cluster_df)} rows). If it's too large, consider truncating.")
                    if len(cluster_df) > max_series_per_cluster:
                        cluster_df = cluster_df.sort_values(by=SERIES_ID_COLS + [DATE_COL])
                        cluster_df = cluster_df.tail(max_series_per_cluster).reset_index(drop=True)
                else:
                    avg_rows_per_series = len(cluster_df) / n_unique_series_original
                    num_series_to_sample = int(max_series_per_cluster / avg_rows_per_series)
                    num_series_to_sample = max(1, min(num_series_to_sample, n_unique_series_original))
                    sampled_series = unique_series.sample(n=num_series_to_sample, random_state=42)
                    cluster_df = cluster_df.merge(sampled_series, on=SERIES_ID_COLS, how='inner')
                    cluster_df = cluster_df.sort_values(by=SERIES_ID_COLS + [DATE_COL]).reset_index(drop=True)
                    print(f"Cluster {cluster_id} now has {len(cluster_df)} rows from {len(sampled_series)} unique series after subsampling.")
            X_cluster = cluster_df[FEATURES_COLS]
            y_cluster = cluster_df[TARGET_VARIABLE]
            ids_cluster = cluster_df[SERIES_ID_COLS + [CLUSTER_COL]]
            dates_cluster = cluster_df[DATE_COL]

            numeric_transformer = Pipeline(steps=[
                ('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())
            ])
            categorical_transformer = Pipeline(steps=[
                ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
                ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=True))
            ])
            preprocessor = ColumnTransformer(
                transformers=[
                    ('num', numeric_transformer, NUMERICAL_FEATURES),
                    ('cat', categorical_transformer, CATEGORICAL_FEATURES_FOR_MODEL)
                ],
                remainder='passthrough'
            )

            print(f"Realizando split Train/Test para el cluster {cluster_id}...")
            test_size_ratio = 0.2
            n_rows = len(cluster_df)
            split_point = int(n_rows * (1 - test_size_ratio))

            if split_point < N_SPLITS_CV + 1 or n_rows - split_point < 1:
                print(f"Datos insuficientes para split Train/Test adecuado en cluster {cluster_id}. Saltando.")
                continue

            X_train_val, X_test = X_cluster.iloc[:split_point], X_cluster.iloc[split_point:]
            y_train_val, y_test = y_cluster.iloc[:split_point], y_cluster.iloc[split_point:]
            ids_test = ids_cluster.iloc[split_point:]
            dates_test = dates_cluster.iloc[split_point:]
            y_true_train_for_mase = y_train_val.copy().values
            print(f"Cluster {cluster_id}: Train/Val={len(X_train_val)}, Test={len(X_test)}")

            cv_test_size = max(1, len(X_train_val) // (N_SPLITS_CV + 1))
            tscv = TimeSeriesSplit(n_splits=N_SPLITS_CV, gap=0, test_size=cv_test_size)

            rf_model = RandomForestRegressor(random_state=42, n_jobs=1)
            pipeline = Pipeline([
                ('preprocess', preprocessor),
                ('regressor', rf_model)
            ])

            search = RandomizedSearchCV(
                estimator=pipeline,
                param_distributions=param_dist_rf,
                n_iter=N_ITER_HPT,
                cv=tscv,
                scoring=SCORING_METRIC_HPT,
                n_jobs=1,
                refit=True,
                random_state=42,
                verbose=1
            )

            print(f"Iniciando HPT con RandomForest para cluster {cluster_id}...")
            search.fit(X_train_val, y_train_val)
            best_model_cluster = search.best_estimator_
            best_params_cluster = search.best_params_
            print(f"Mejores parámetros encontrados para cluster {cluster_id} (RandomForest): {best_params_cluster}")

            print(f"Realizando predicciones en el conjunto de test del cluster {cluster_id}...")
            y_pred_test_cluster = best_model_cluster.predict(X_test)

            print(f"Calculando métricas agregadas para el cluster {cluster_id}...")
            cluster_metrics = evaluate_model(y_true_train_for_mase, y_test.values, y_pred_test_cluster, label="Cluster")
            metrics_row = {CLUSTER_COL: cluster_id}
            metrics_row.update(cluster_metrics)
            metrics_row['best_params'] = str(best_params_cluster)
            pd.DataFrame([metrics_row]).to_csv(metrics_file, mode='a', header=False, index=False)

            predictions_df = pd.DataFrame({
                'establecimiento': ids_test['establecimiento'],
                'material': ids_test['material'],
                DATE_COL: dates_test,
                'actual_volume': y_test.values,
                'predicted_volume': y_pred_test_cluster,
                CLUSTER_COL: ids_test[CLUSTER_COL]
            })
            predictions_df.to_csv(predictions_file, mode='a', header=False, index=False)

            print(f"Generando gráficos para las series del cluster {cluster_id}...")
            unique_series_in_test_df = ids_test[SERIES_ID_COLS].drop_duplicates()
            plot_count = 0
            for _, series_row in unique_series_in_test_df.iterrows():
                if plot_count >= max_plots_per_cluster:
                    print(f"Límite de {max_plots_per_cluster} gráficos alcanzado para el cluster {cluster_id}.")
                    break
                estab = series_row['establecimiento']
                material = series_row['material']
                mask = (predictions_df['establecimiento'] == estab) & \
                       (predictions_df['material'] == material)
                series_pred_df = predictions_df[mask]
                if not series_pred_df.empty:
                    plot_filename = os.path.join(plots_dir, f'pred_vs_actual_rf_{estab}_{material}_cluster{cluster_id}.png')
                    plot_predictions(
                        series_pred_df[DATE_COL],
                        series_pred_df['actual_volume'],
                        series_pred_df['predicted_volume'],
                        estab, material, cluster_id, plot_filename
                    )
                    plot_count += 1

            model_filename = os.path.join(models_dir, f'rf_model_cluster_{cluster_id}.joblib')
            joblib.dump(best_model_cluster, model_filename)
            print(f"Modelo RandomForest del cluster {cluster_id} guardado en: {model_filename}")

        except Exception as e:
            print(f"ERROR procesando el cluster {cluster_id}: {e}")
            import traceback
            traceback.print_exc()
            error_row = {CLUSTER_COL: cluster_id}
            for m_key in header_metrics:
                if m_key not in [CLUSTER_COL, 'best_params']:
                    error_row[m_key] = 'ERROR'
            error_row['best_params'] = str(e)
            pd.DataFrame([error_row]).to_csv(metrics_file, mode='a', header=False, index=False)

        finally:
            gc.collect()

    print("\n--- Proceso Completado ---")
    print(f"Resultados de RandomForest guardados en el directorio: {output_dir}")

# Ejemplo de uso:
# cluster_labels = [1, 2, 3]  # O cargar desde archivo/metadata
# run_random_forest_by_clusters(cluster_labels)

In [20]:
run_random_forest_by_clusters([6])


Iniciando procesamiento para 1 clusters...

--- Procesando Cluster 1/1: ID = 6 ---
Cargando datos para el cluster 6...
Cluster 6: 941142 filas cargadas.
Cluster 6 is very large (941142 rows). Subsampling complete series to be around 500000 rows.
Cluster 6 now has 497618 rows from 3835 unique series after subsampling.
Realizando split Train/Test para el cluster 6...
Cluster 6: Train/Val=398094, Test=99524
Iniciando HPT con RandomForest para cluster 6...
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Mejores parámetros encontrados para cluster 6 (RandomForest): {'regressor__n_estimators': 100, 'regressor__min_samples_split': 100, 'regressor__min_samples_leaf': 10, 'regressor__max_features': 0.7, 'regressor__max_depth': 20}
Realizando predicciones en el conjunto de test del cluster 6...
Calculando métricas agregadas para el cluster 6...
Generando gráficos para las series del cluster 6...
Límite de 100 gráficos alcanzado para el cluster 6.
Modelo RandomForest del cluster 6 gu